In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Introduction

Because of the relational structure in a graph,
we can begin to think about "importance" of a node
that is induced because of its relationships
to the rest of the nodes in the graph.

Before we go on, let's think about
a pertinent and contemporary example.

### An example: contact tracing

At the time of writing (April 2020),
finding important nodes in a graph has actually taken on a measure of importance
that we might not have appreciated before.
With the COVID-19 virus spreading,
contact tracing has become quite important.
In an infectious disease contact network,
where individuals are nodes and
contact between individuals of some kind are the edges,
an "important" node in this contact network
would be an individual who was infected
who also was in contact with many people
during the time that they were infected.

### Our dataset: "Sociopatterns"

The dataset that we will use in this chapter is the "[sociopatterns network][sociopatterns]" dataset.
Incidentally, it's also about infectious diseases.

[sociopatterns]: http://konect.uni-koblenz.de/networks/sociopatterns-infectious

Here is the description of the dataset.

> This network describes the face-to-face behavior of people
> during the exhibition INFECTIOUS: STAY AWAY in 2009
> at the Science Gallery in Dublin.
> Nodes represent exhibition visitors;
> edges represent face-to-face contacts that were active for at least 20 seconds.
> Multiple edges between two nodes are possible and denote multiple contacts.
> The network contains the data from the day with the most interactions.

To simplify the network, we have represented only the last contact between individuals.

In [3]:
from nams import load_data as cf
G = cf.load_sociopatterns_network()

It is loaded as an undirected graph object:

In [5]:
type(G)

networkx.classes.graph.Graph

As usual, before proceeding with any analysis,
we should know basic graph statistics.

In [6]:
len(G.nodes()), len(G.edges())

(410, 2765)

## A Measure of Importance: "Number of Neighbors"

One measure of importance of a node is
the number of **neighbors** that the node has.
What is a **neighbor**?
We will work with the following definition:

> The neighbor of a node is connected to that node by an edge.

Let's explore this concept, using the NetworkX API.

Every NetworkX graph provides a `G.neighbors(node)` class method,
which lets us query a graph for the number of neighbors
of a given node:

In [11]:
G.neighbors(7)

It returns a generator that doesn't immediately return
the exact neighbors list.
This means we cannot know its exact length,
as it is a generator.
If you tried to do:

```python
len(G.neighbors(7))
```

you would get the following error:

```python
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-13-72c56971d077> in <module>
----> 1 len(G.neighbors(7))

TypeError: object of type 'dict_keyiterator' has no len()
```

Hence, we will need to cast it as a list in order to know
both its length
and its members:

In [14]:
list(G.neighbors(7))

[5, 6, 21, 22, 37, 48, 51]

In the event that some nodes have an extensive list of neighbors,
then using the `dict_keyiterator` is potentially a good memory-saving technique,
as it lazily yields the neighbors.

### Exercise: Rank-ordering the number of neighbors a node has

Since we know how to get the list of nodes that are neighbors of a given node,
try this following exercise:

> Can you create a ranked list of the importance of each individual, based on the number of neighbors they have?

Here are a few hints to help:

- You could consider using a `pandas Series`. This would be a modern and idiomatic way of approaching the problem.
- You could also consider using Python's `sorted` function.

In [42]:
from nams.solutions.hubs import rank_ordered_neighbors

#### REPLACE THE NEXT FEW LINES WITH YOUR ANSWER
# answer = rank_ordered_neighbors(G)
# answer

The original implementation looked like the following

In [19]:
from nams.solutions.hubs import rank_ordered_neighbors_original
# rank_ordered_neighbors_original??

And another implementation that uses generators:

In [21]:
from nams.solutions.hubs import rank_ordered_neighbors_generator
# rank_ordered_neighbors_generator??

## Generalizing "neighbors" to arbitrarily-sized graphs

The concept of neighbors is simple and appealing,
but it leaves us with a slight point of dissatisfaction:
it is difficult to compare graphs of different sizes.
Is a node more important solely because it has more neighbors?
What if it were situated in an extremely large graph?
Would we not expect it to have more neighbors?

As such, we need a normalization factor.
One reasonable one, in fact, is
_the number of nodes that a given node could **possibly** be connected to._
By taking the ratio of the number of neighbors a node has
to the number of neighbors it could possibly have,
we get the **degree centrality** metric.

Formally defined, the degree centrality of a node (let's call it $d$)
is the number of neighbors that a node has (let's call it $n$)
divided by the number of neighbors it could _possibly_ have (let's call it $N$):

$$d = \frac{n}{N}$$

NetworkX provides a function for us to calculate degree centrality conveniently:

In [26]:
import networkx as nx
import pandas as pd
dcs = pd.Series(nx.degree_centrality(G))
dcs

100    0.070905
101    0.031785
102    0.039120
103    0.063570
104    0.041565
         ...   
89     0.009780
91     0.051345
96     0.036675
99     0.034230
98     0.002445
Length: 410, dtype: float64

`nx.degree_centrality(G)` returns to us a dictionary of key-value pairs,
where the keys are node IDs
and values are the degree centrality score.
To save on output length, I took the liberty of casting it as a pandas Series
to make it easier to display.

Incidentally, we can also sort the series
to find the nodes with the highest degree centralities:

In [27]:
dcs.sort_values(ascending=False)

51     0.122249
272    0.114914
195    0.105134
235    0.105134
161    0.083130
         ...   
119    0.002445
79     0.002445
135    0.002445
313    0.002445
98     0.002445
Length: 410, dtype: float64

Does the list order look familiar?
It should, since the numerator of the degree centrality metric
is identical to the number of neighbors,
and the denominator is a constant.

### Distribution of graph metrics

One important concept that you should come to know
is that the distribution of node-centric values
can characterize classes of graphs.

What do we mean by "distribution of node-centric values"?
One would be the degree distribution,
that is, the collection of node degree values in a graph.

Generally, you might be familiar with plotting a histogram
to visualize distributions of values,
but in this book, we are going to avoid histograms like the plague.
I detail a lot of reasons in a [blog post][ecdf] I wrote in 2018,
but the main points are that:

1. It's easier to lie with histograms.
1. You get informative statistical information (median, IQR, extremes/outliers)
more easily.

[ecdf]: https://ericmjl.github.io/blog/2018/7/14/ecdfs/

### Exercise: Degree distribution

In this next exercise, we are going to get practice visualizing these values
using empirical cumulative distribution function plots.

I have written for you an ECDF function that you can use already.
Its API looks like the following:

```python
x, y = ecdf(list_of_values)
```

giving you `x` and `y` values that you can directly plot.

The exercise prompt is this:

> Plot the ECDF of the degree centrality and degree distributions.

First do it for **degree centrality**:

In [43]:
from nams.functions import ecdf
from nams.solutions.hubs import ecdf_degree_centrality

#### REPLACE THE FUNCTION CALL WITH YOUR ANSWER
# ecdf_degree_centrality(G)

Now do it for **degree**:

In [44]:
from nams.solutions.hubs import ecdf_degree

#### REPLACE THE FUNCTION CALL WITH YOUR ANSWER
# ecdf_degree(G)

The fact that they are identically-shaped
should not surprise you!

### Exercise: What about that denominator?

The denominator $N$ in the degree centrality definition
is "the number of nodes that a node could _possibly_ be connected to".
Can you think of two ways $N$ be defined?

In [45]:
from nams.solutions.hubs import num_possible_neighbors

#### UNCOMMENT TO SEE MY ANSWER
# print(num_possible_neighbors())

## Solutions

Here are the solutions to the exercises above.

In [46]:
from nams.solutions import hubs
import inspect

print(inspect.getsource(hubs))

import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd

from nams import ecdf


def rank_ordered_neighbors(G):
	"""
	Uses a pandas Series to help with sorting.
	"""
	s = pd.Series(
			{
				n: len(list(G.neighbors(n)))
            	for n
            	in G.nodes()
        	}
    	)
	return s.sort_values(ascending=False)


def rank_ordered_neighbors_original(G):
	return sorted(G.nodes(),
       key=lambda x:len(list(G.neighbors(x))), reverse=True)


def rank_ordered_neighbors_generator(G):
	"""
	Contributed by @dgerlanc.

	Ref: https://github.com/ericmjl/Network-Analysis-Made-Simple/issues/75
	"""
	gen = ((len(list(G.neighbors(x))), x) for x in G.nodes())
	return sorted(gen, reverse=True)


def ecdf_degree_centrality(G):
	x, y = ecdf(list(nx.degree_centrality(G).values()))
	plt.scatter(x, y)
	plt.xlabel("degree centrality")
	plt.ylabel("cumulative fraction")

def ecdf_degree(G):
	num_neighbors = [len(list(G.neighbors(n))) for n in G.nodes()]
	x, y = ecdf(num_neighbors)